In [1]:
import transformers
import pandas as pd

from datasets import load_dataset
from app.model import IntentClassifier


dataset = load_dataset("tuetschek/atis")

Generating train split:   0%|          | 0/4978 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/893 [00:00<?, ? examples/s]

In [2]:
from app.atis.utils import ATIS_INTENT_MAPPING as intent_mapping
intent_mapping

{'abbreviation': 'Abbreviation and Fare Code Meaning Inquiry',
 'aircraft': 'Aircraft Type Inquiry',
 'airfare': 'Airfare Information Requests',
 'airline': 'Airline Information Request',
 'airport': 'Airport Information and Queries',
 'capacity': 'Aircraft Seating Capacity Inquiry',
 'cheapest': 'Cheapest Fare Inquiry',
 'city': 'Airport Location Inquiry',
 'distance': 'Airport Distance Inquiry',
 'flight': 'Flight Booking Request',
 'flight_no': 'Flight Number Inquiry',
 'flight_time': 'Flight Schedule Inquiry',
 'ground_fare': 'Ground Transportation Cost Inquiry',
 'ground_service': 'Ground Transportation Inquiry',
 'ground_service+ground_fare': 'Airport Ground Transportation and Cost Query',
 'meal': 'Inquiry about In-flight Meals',
 'quantity': 'Flight Quantity Inquiry',
 'restriction': 'Flight Restriction Inquiry'}

In [24]:
from tqdm import tqdm
import datasets

results = []
test_set = []
clean_test_set = []

labels = list(intent_mapping.values())
label_to_id = {}
for i in range(len(labels)):
    label_to_id[labels[i]] = i
    
for row in tqdm(dataset["test"]):
    intent = row["intent"] 
    if intent not in intent_mapping:
        continue 
    # row["intent"] = intent_mapping[row["intent"]]
    test_set.append(row)
    new_name = intent_mapping[row["intent"]]
    clean_test_set.append({"label": label_to_id[new_name], "text": row["text"], "label_name": new_name})

test_df = pd.DataFrame(data=clean_test_set)
test_dataset = datasets.Dataset.from_pandas(test_df)
test_set[0]

100%|██████████| 893/893 [00:00<00:00, 17107.49it/s]


{'id': 0,
 'intent': 'flight',
 'text': 'i would like to find a flight from charlotte to las vegas that makes a stop in st. louis',
 'slots': 'O O O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O O O O O B-stoploc.city_name I-stoploc.city_name'}

## Train Setfit Zero shot 


In [14]:
from setfit import get_templated_dataset

print(labels)
train_dataset = get_templated_dataset(candidate_labels=labels, sample_size=8)
print(train_dataset[0])



['Abbreviation and Fare Code Meaning Inquiry', 'Aircraft Type Inquiry', 'Airfare Information Requests', 'Airline Information Request', 'Airport Information and Queries', 'Aircraft Seating Capacity Inquiry', 'Cheapest Fare Inquiry', 'Airport Location Inquiry', 'Airport Distance Inquiry', 'Flight Booking Request', 'Flight Number Inquiry', 'Flight Schedule Inquiry', 'Ground Transportation Cost Inquiry', 'Ground Transportation Inquiry', 'Airport Ground Transportation and Cost Query', 'Inquiry about In-flight Meals', 'Flight Quantity Inquiry', 'Flight Restriction Inquiry']
{'text': 'This sentence is Abbreviation and Fare Code Meaning Inquiry', 'label': 0}


In [15]:
from setfit import SetFitModel, Trainer, TrainingArguments

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")

args = TrainingArguments(
    batch_size=32,
    num_epochs=1,
    num_iterations=1,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()


/Users/sergey/dev/loris/original_t5_intent_classification/venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/144 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 288
  Batch size = 32
  Num epochs = 1
  Total optimization steps = 9


Step,Training Loss


In [16]:
metrics = trainer.evaluate()
print(metrics)

***** Running evaluation *****


{'accuracy': 0.5821917808219178}


In [39]:
model.predict("Hey, how can I get a flight?")

tensor(9)

In [21]:
from datasets import load_metric
metric = load_metric("f1")
model_predictions = model(test_df["text"].tolist())
final_score = metric.compute(predictions=model_predictions, references=test_df["label"], average="weighted")


/Users/sergey/dev/loris/original_t5_intent_classification/venv/lib/python3.9/site-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [36]:
from sklearn.metrics import classification_report
print(final_score)
predictions = model_predictions.cpu().numpy()
# print(predictions)
report = classification_report(test_df["label"], predictions, output_dict=True)
df = pd.DataFrame(report).transpose()
df


{'f1': 0.6879956440691523}


/Users/sergey/dev/loris/original_t5_intent_classification/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/dev/loris/original_t5_intent_classification/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/dev/loris/original_t5_intent_classification/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to 

,precision,recall,f1-score,support
0,1.000000,0.787879,0.881356,33.000000
1,1.000000,0.555556,0.714286,9.000000
2,0.625000,0.104167,0.178571,48.000000
3,0.818182,0.710526,0.760563,38.000000
4,0.229730,0.944444,0.369565,18.000000
5,0.840000,1.000000,0.913043,21.000000
6,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,6.000000
8,0.818182,0.900000,0.857143,10.000000
9,0.984127,0.588608,0.736634,632.000000


## Lets try a differrent sentence transformers model

In [40]:
from setfit import SetFitModel, Trainer, TrainingArguments

model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

args = TrainingArguments(
    batch_size=32,
    num_epochs=1,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()



config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/144 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 19584
  Batch size = 32
  Num epochs = 1
  Total optimization steps = 612


Step,Training Loss


In [37]:
metrics = trainer.evaluate()
print(metrics)

***** Running evaluation *****


{'accuracy': 0.5821917808219178}
